# Notebook para Gerar dataframe sem os classificados como desistentes depois de cada treinamento com o Random Forest

## Importando as libs usadas no processo

In [86]:
import warnings
warnings.filterwarnings('ignore')

import itertools

import pandas as pd
import numpy as np

from matplotlib import pyplot as plt
from pprint import pprint


from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
#from sklearn.metrics import plot_confusion_matrix
#from sklearn.metrics import precision_score


from sklearn.model_selection import KFold
#from sklearn.model_selection import cross_val_score
#from sklearn.model_selection import cross_validate
#from sklearn.model_selection import GridSearchCV
#from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier

from xgboost.sklearn import XGBClassifier

from sklearn.naive_bayes import GaussianNB

## Impontando os arquivos para gerar os DataFrames usados

Os aquivos estão em formato csv e foram gerados a partir de scripts de extração de dados, cada CSV corresponde aos dados acumulados até uma prova N. Além do CSV que contém os usuários e sua classificação.

Eles foram importados e transformados em DataFrame pandas

In [87]:
# import datas
df1 = pd.read_csv('/content/drive/Shared drives/iniciacao_cientifica_2019_2020/dataframes_treino/turma_102_2016_1/Dataframe1_Turma_102_2016_06_07_12_00_00_2016_06_30_11_55_00.csv', sep=';')
df2 = pd.read_csv('/content/drive/Shared drives/iniciacao_cientifica_2019_2020/dataframes_treino/turma_102_2016_1/Dataframe2_Turma_102_2016_06_07_12_00_00_2016_07_19_13_20_00.csv', sep=';')
df3 = pd.read_csv('/content/drive/Shared drives/iniciacao_cientifica_2019_2020/dataframes_treino/turma_102_2016_1/Dataframe3_Turma_102_2016_06_07_12_00_00_2016_07_28_12_00_00.csv', sep=';')
df4 = pd.read_csv('/content/drive/Shared drives/iniciacao_cientifica_2019_2020/dataframes_treino/turma_102_2016_1/Dataframe4_Turma_102_2016_06_07_12_00_00_2016_08_11_11_55_00.csv', sep=';')
df5 = pd.read_csv('/content/drive/Shared drives/iniciacao_cientifica_2019_2020/dataframes_treino/turma_102_2016_1/Dataframe5_Turma_102_2016_06_07_12_00_00_2016_08_25_11_55_00.csv', sep=';')
df6 = pd.read_csv('/content/drive/Shared drives/iniciacao_cientifica_2019_2020/dataframes_treino/turma_102_2016_1/Dataframe6_Turma_102_2016_06_07_12_00_00_2016_09_13_12_45_00.csv', sep=';')

df_target_user = pd.read_csv('/content/drive/Shared drives/iniciacao_cientifica_2019_2020/csv_datasets/usuarios_classificados.csv')
df_target_user = df_target_user[['semester_id', 'class_id', 'user_id', 'target']]

df1_data_target = pd.merge(df1, df_target_user, on=['semester_id', 'class_id', 'user_id'], how='inner')
df2_data_target = pd.merge(df2, df_target_user, on=['semester_id', 'class_id', 'user_id'], how='inner')
df3_data_target = pd.merge(df3, df_target_user, on=['semester_id', 'class_id', 'user_id'], how='inner')
df4_data_target = pd.merge(df4, df_target_user, on=['semester_id', 'class_id', 'user_id'], how='inner')
df5_data_target = pd.merge(df5, df_target_user, on=['semester_id', 'class_id', 'user_id'], how='inner')
df6_data_target = pd.merge(df6, df_target_user, on=['semester_id', 'class_id', 'user_id'], how='inner')

df1_data_target.drop(['semester_id', 'class_id', 'user_id'], axis=1, inplace=True)    
df2_data_target.drop(['semester_id', 'class_id', 'user_id'], axis=1, inplace=True) 
df3_data_target.drop(['semester_id', 'class_id', 'user_id'], axis=1, inplace=True) 
df4_data_target.drop(['semester_id', 'class_id', 'user_id'], axis=1, inplace=True) 
df5_data_target.drop(['semester_id', 'class_id', 'user_id'], axis=1, inplace=True) 
df6_data_target.drop(['semester_id', 'class_id', 'user_id'], axis=1, inplace=True) 


## Separando os dados treino até 1º prova

In [88]:
# get the features antil the first exam
features = df1_data_target.columns.difference(['target'])

# train data
X = df1_data_target[features].values

# target
Y = df1_data_target['target'].values

# creating a dataframe wiht all preditions of random forest
df_rf_all_predictions = pd.DataFrame()
df_rf_all_predictions['target'] = Y

df_etc_all_predictions = pd.DataFrame()
df_etc_all_predictions['target'] = Y

df_xgb_all_predictions = pd.DataFrame()
df_xgb_all_predictions['target'] = Y

df_gnb_all_predictions = pd.DataFrame()
df_gnb_all_predictions['target'] = Y

## Treinando com os dados até a 1º prova

In [89]:

acur_rf = []
acur_etc = []
acur_xgb = []
acur_gnb = []

dict_predict_rf = {}
dict_predict_etc = {}
dict_predict_xgb = {}
dict_predict_gnb = {}

# model
classifier_rf = RandomForestClassifier(bootstrap=False, criterion='entropy', max_depth=5, n_estimators=25, verbose=0)
classifier_etc = ExtraTreesClassifier(bootstrap=False, criterion='entropy', max_depth=3, n_estimators=25)
classifier_xgb = XGBClassifier(learning_rate=0.1, max_depth=2, n_estimators=60)
classifier_gnb = GaussianNB()

# cv = KFold(n_splits=10,  random_state=2**32 - 1, shuffle=True)
cv = KFold(n_splits=10, shuffle=True)
for train_index, test_index in cv.split(X):

    X_train, X_test, y_train, y_test = X[train_index], X[test_index], Y[train_index], Y[test_index]

    # criando um particao de validação para o modelo xgb
    X_val, y_val = X_test[0:2], y_test[0:2]
    _X_test = X_test[2:]
    _y_test = y_test[2:]

    # fit model
    classifier_rf.fit(X_train, y_train)
    classifier_etc.fit(X_train, y_train)
    classifier_gnb.fit(X_train, y_train)
    classifier_xgb.fit(X_train, y_train, early_stopping_rounds=10, eval_set=[(X_val, y_val)])

    # testing model
    y_predict_rf  = classifier_rf.predict(X_test)
    y_predict_etc = classifier_etc.predict(X_test)
    y_predict_xgb = classifier_xgb.predict(X_test)
    y_predict_gnb = classifier_gnb.predict(X_test)

    # dataframe predictions
    d = dict(zip(test_index, y_predict_rf))
    dict_predict_rf.update(d)
    d = dict(zip(test_index, y_predict_etc))
    dict_predict_etc.update(d)
    d = dict(zip(test_index, y_predict_xgb))
    dict_predict_xgb.update(d)
    d = dict(zip(test_index, y_predict_gnb))
    dict_predict_gnb.update(d)

    # acum accurary
    acur_rf.append(accuracy_score(y_test, y_predict_rf))
    acur_etc.append(accuracy_score(y_test, y_predict_etc))
    acur_xgb.append(accuracy_score(y_test, y_predict_xgb))
    acur_gnb.append(accuracy_score(y_test, y_predict_gnb))
    
    
print('Acurácia Média Random Forest:', np.mean(acur_rf))
print('Acurácia Média Extra Tree Classifier:', np.mean(acur_etc))
print('Acurácia Média XGBoost', np.mean(acur_xgb))
print('Acurácia Média Gaussian Naive Bayes', np.mean(acur_gnb))

d = dict(sorted(dict_predict_rf.items()))
a = np.array([w for v,w in d.items()])
df_rf_all_predictions['exam_1'] = a

d = dict(sorted(dict_predict_etc.items()))
a = np.array([w for v,w in d.items()])
df_etc_all_predictions['exam_1'] = a

d = dict(sorted(dict_predict_xgb.items()))
a = np.array([w for v,w in d.items()])
df_xgb_all_predictions['exam_1'] = a

d = dict(sorted(dict_predict_gnb.items()))
a = np.array([w for v,w in d.items()])
df_gnb_all_predictions['exam_1'] = a


[0]	validation_0-error:0
Will train until validation_0-error hasn't improved in 10 rounds.
[1]	validation_0-error:0
[2]	validation_0-error:0
[3]	validation_0-error:0
[4]	validation_0-error:0
[5]	validation_0-error:0
[6]	validation_0-error:0
[7]	validation_0-error:0
[8]	validation_0-error:0
[9]	validation_0-error:0
[10]	validation_0-error:0
Stopping. Best iteration:
[0]	validation_0-error:0

[0]	validation_0-error:0
Will train until validation_0-error hasn't improved in 10 rounds.
[1]	validation_0-error:0
[2]	validation_0-error:0
[3]	validation_0-error:0
[4]	validation_0-error:0
[5]	validation_0-error:0
[6]	validation_0-error:0
[7]	validation_0-error:0
[8]	validation_0-error:0
[9]	validation_0-error:0
[10]	validation_0-error:0
Stopping. Best iteration:
[0]	validation_0-error:0

[0]	validation_0-error:0
Will train until validation_0-error hasn't improved in 10 rounds.
[1]	validation_0-error:0
[2]	validation_0-error:0
[3]	validation_0-error:0
[4]	validation_0-error:0
[5]	validation_0-erro

## Treinando o modelo com dados até a 2º prova e sem os desistentes

## Random Forest

In [90]:
i = df_rf_all_predictions.loc[df_rf_all_predictions['exam_1'] == 'Não Desistente'].index
df2_ = df2_data_target.iloc[i].copy()
X = df2_[features].values
Y = df2_['target'].values

# traning Random Forest
acur_rf = []
dict_predict_rf = {}

# model
classifier_rf = RandomForestClassifier(bootstrap=False, criterion='entropy', max_depth=5, n_estimators=25, verbose=0)

# cv = KFold(n_splits=10,  random_state=2**32 - 1, shuffle=True)
cv = KFold(n_splits=10, shuffle=True)
for train_index, test_index in cv.split(X):

    X_train, X_test, y_train, y_test = X[train_index], X[test_index], Y[train_index], Y[test_index]

    # fit model
    classifier_rf.fit(X_train, y_train)

    # testing model
    y_predict_rf  = classifier_rf.predict(X_test)

    # dataframe predictions
    d = dict(zip(test_index, y_predict_rf))
    dict_predict_rf.update(d)

    # acum accurary
    acur_rf.append(accuracy_score(y_test, y_predict_rf))
    
print('Acurácia Média Random Forest:', np.mean(acur_rf))

d = dict(sorted(dict_predict_rf.items()))
a = np.array([w for v,w in d.items()])
df2_['exam_2'] = a

df_rf_all_predictions = df_rf_all_predictions.join(df2_[['exam_2']])


Acurácia Média Random Forest: 0.975


## Extra Tree Classifier

In [91]:
i = df_etc_all_predictions.loc[df_etc_all_predictions['exam_1'] == 'Não Desistente'].index
df2_ = df2_data_target.iloc[i].copy()
X = df2_[features].values
Y = df2_['target'].values

acur_etc = []

dict_predict_etc = {}

# model
classifier_etc = ExtraTreesClassifier(bootstrap=False, criterion='entropy', max_depth=3, n_estimators=25)

# cv = KFold(n_splits=10,  random_state=2**32 - 1, shuffle=True)
cv = KFold(n_splits=10, shuffle=True)
for train_index, test_index in cv.split(X):

    X_train, X_test, y_train, y_test = X[train_index], X[test_index], Y[train_index], Y[test_index]

    # fit model
    classifier_etc.fit(X_train, y_train)

    # testing model
    y_predict_etc = classifier_etc.predict(X_test)

    # dataframe predictions
    d = dict(zip(test_index, y_predict_etc))
    dict_predict_etc.update(d)

    # acum accurary
    acur_etc.append(accuracy_score(y_test, y_predict_etc))
    
print('Acurácia Média Extra Tree Classifier:', np.mean(acur_etc))

d = dict(sorted(dict_predict_etc.items()))
a = np.array([w for v,w in d.items()])
df2_['exam_2'] = a

df_etc_all_predictions = df_etc_all_predictions.join(df2_[['exam_2']])


Acurácia Média Extra Tree Classifier: 1.0


### XGBoost

In [92]:
i = df_xgb_all_predictions.loc[df_xgb_all_predictions['exam_1'] == 'Não Desistente'].index
df2_ = df2_data_target.iloc[i].copy()
X = df2_[features].values
Y = df2_['target'].values

acur_xgb = []

dict_predict_xgb = {}

# model
classifier_xgb = XGBClassifier(learning_rate=0.1, max_depth=2, n_estimators=60)

# cv = KFold(n_splits=10,  random_state=2**32 - 1, shuffle=True)
cv = KFold(n_splits=10, shuffle=True)
for train_index, test_index in cv.split(X):

    X_train, X_test, y_train, y_test = X[train_index], X[test_index], Y[train_index], Y[test_index]

    # criando um particao de validação para o modelo xgb
    X_val, y_val = X_test[0:2], y_test[0:2]
    _X_test = X_test[2:]
    _y_test = y_test[2:]

    # fit model
    classifier_xgb.fit(X_train, y_train, early_stopping_rounds=10, eval_set=[(X_val, y_val)])

    # testing model
    y_predict_xgb = classifier_xgb.predict(X_test)

    # dataframe predictions
    d = dict(zip(test_index, y_predict_xgb))
    dict_predict_xgb.update(d)

    # acum accurary
    acur_xgb.append(accuracy_score(y_test, y_predict_xgb))
    
print('Acurácia Média XGBoost', np.mean(acur_xgb))

d = dict(sorted(dict_predict_xgb.items()))
a = np.array([w for v,w in d.items()])
df2_['exam_2'] = a

df_xgb_all_predictions = df_xgb_all_predictions.join(df2_[['exam_2']])


[0]	validation_0-error:0
Will train until validation_0-error hasn't improved in 10 rounds.
[1]	validation_0-error:0
[2]	validation_0-error:0
[3]	validation_0-error:0
[4]	validation_0-error:0
[5]	validation_0-error:0
[6]	validation_0-error:0
[7]	validation_0-error:0
[8]	validation_0-error:0
[9]	validation_0-error:0
[10]	validation_0-error:0
Stopping. Best iteration:
[0]	validation_0-error:0

[0]	validation_0-error:0.5
Will train until validation_0-error hasn't improved in 10 rounds.
[1]	validation_0-error:0.5
[2]	validation_0-error:0.5
[3]	validation_0-error:0.5
[4]	validation_0-error:0.5
[5]	validation_0-error:0.5
[6]	validation_0-error:0
[7]	validation_0-error:0.5
[8]	validation_0-error:0
[9]	validation_0-error:0
[10]	validation_0-error:0
[11]	validation_0-error:0
[12]	validation_0-error:0
[13]	validation_0-error:0
[14]	validation_0-error:0
[15]	validation_0-error:0
[16]	validation_0-error:0
Stopping. Best iteration:
[6]	validation_0-error:0

[0]	validation_0-error:0
Will train until 

### Gaussian Naive Bayes

In [93]:
i = df_gnb_all_predictions.loc[df_gnb_all_predictions['exam_1'] == 'Não Desistente'].index
df2_ = df2_data_target.iloc[i].copy()
X = df2_[features].values
Y = df2_['target'].values

acur_gnb = []

dict_predict_gnb = {}

# model
classifier_gnb = GaussianNB()

# cv = KFold(n_splits=10,  random_state=2**32 - 1, shuffle=True)
cv = KFold(n_splits=10, shuffle=True)
for train_index, test_index in cv.split(X):

    X_train, X_test, y_train, y_test = X[train_index], X[test_index], Y[train_index], Y[test_index]

    # fit model
    classifier_gnb.fit(X_train, y_train)

    # testing model
    y_predict_gnb = classifier_gnb.predict(X_test)

    # dataframe predictions
    d = dict(zip(test_index, y_predict_gnb))
    dict_predict_gnb.update(d)

    # acum accurary
    acur_gnb.append(accuracy_score(y_test, y_predict_gnb))

print('Acurácia Média Gaussian Naive Bayes', np.mean(acur_gnb))

d = dict(sorted(dict_predict_gnb.items()))
a = np.array([w for v,w in d.items()])
df2_['exam_2'] = a

df_gnb_all_predictions = df_gnb_all_predictions.join(df2_[['exam_2']])


Acurácia Média Gaussian Naive Bayes 0.95


## Treinando os com dados até 3º prova

### Random Forest

In [94]:
# selecting all não desistente
i = df_rf_all_predictions.loc[df_rf_all_predictions['exam_2'] == 'Não Desistente'].index
df3_ = df3_data_target.iloc[i].copy()
X = df3_[features].values
Y = df3_['target'].values

# traning Random Forest
acur_rf = []
dict_predict_rf = {}

# model
classifier_rf = RandomForestClassifier(bootstrap=False, criterion='entropy', max_depth=5, n_estimators=25, verbose=0)

# cv = KFold(n_splits=10,  random_state=2**32 - 1, shuffle=True)
cv = KFold(n_splits=10, shuffle=True)
for train_index, test_index in cv.split(X):

    X_train, X_test, y_train, y_test = X[train_index], X[test_index], Y[train_index], Y[test_index]

    # fit model
    classifier_rf.fit(X_train, y_train)

    # testing model
    y_predict_rf  = classifier_rf.predict(X_test)

    # dataframe predictions
    d = dict(zip(test_index, y_predict_rf))
    dict_predict_rf.update(d)

    # acum accurary
    acur_rf.append(accuracy_score(y_test, y_predict_rf))
    
print('Acurácia Média Random Forest:', np.mean(acur_rf))

d = dict(sorted(dict_predict_rf.items()))
a = np.array([w for v,w in d.items()])
df3_['exam_3'] = a

df_rf_all_predictions = df_rf_all_predictions.join(df3_[['exam_3']])


Acurácia Média Random Forest: 0.975


### Extra Tree Classifier

In [95]:
i = df_etc_all_predictions.loc[df_etc_all_predictions['exam_2'] == 'Não Desistente'].index
df3_ = df3_data_target.iloc[i].copy()
X = df3_[features].values
Y = df3_['target'].values

acur_etc = []

dict_predict_etc = {}

# model
classifier_etc = ExtraTreesClassifier(bootstrap=False, criterion='entropy', max_depth=3, n_estimators=25)

# cv = KFold(n_splits=10,  random_state=2**32 - 1, shuffle=True)
cv = KFold(n_splits=10, shuffle=True)
for train_index, test_index in cv.split(X):

    X_train, X_test, y_train, y_test = X[train_index], X[test_index], Y[train_index], Y[test_index]

    # fit model
    classifier_etc.fit(X_train, y_train)

    # testing model
    y_predict_etc = classifier_etc.predict(X_test)

    # dataframe predictions
    d = dict(zip(test_index, y_predict_etc))
    dict_predict_etc.update(d)

    # acum accurary
    acur_etc.append(accuracy_score(y_test, y_predict_etc))
    
print('Acurácia Média Extra Tree Classifier:', np.mean(acur_etc))

d = dict(sorted(dict_predict_etc.items()))
a = np.array([w for v,w in d.items()])
df3_['exam_3'] = a

df_etc_all_predictions = df_etc_all_predictions.join(df3_[['exam_3']])


Acurácia Média Extra Tree Classifier: 1.0


### XGBoost

In [96]:
i = df_xgb_all_predictions.loc[df_xgb_all_predictions['exam_2'] == 'Não Desistente'].index
df3_ = df3_data_target.iloc[i].copy()
X = df3_[features].values
Y = df3_['target'].values

acur_xgb = []

dict_predict_xgb = {}

# model
classifier_xgb = XGBClassifier(learning_rate=0.1, max_depth=2, n_estimators=60)

# cv = KFold(n_splits=10,  random_state=2**32 - 1, shuffle=True)
cv = KFold(n_splits=10, shuffle=True)
for train_index, test_index in cv.split(X):

    X_train, X_test, y_train, y_test = X[train_index], X[test_index], Y[train_index], Y[test_index]

    # criando um particao de validação para o modelo xgb
    X_val, y_val = X_test[0:2], y_test[0:2]
    _X_test = X_test[2:]
    _y_test = y_test[2:]

    # fit model
    classifier_xgb.fit(X_train, y_train, early_stopping_rounds=10, eval_set=[(X_val, y_val)])

    # testing model
    y_predict_xgb = classifier_xgb.predict(X_test)

    # dataframe predictions
    d = dict(zip(test_index, y_predict_xgb))
    dict_predict_xgb.update(d)

    # acum accurary
    acur_xgb.append(accuracy_score(y_test, y_predict_xgb))
    
print('Acurácia Média XGBoost', np.mean(acur_xgb))

d = dict(sorted(dict_predict_xgb.items()))
a = np.array([w for v,w in d.items()])
df3_['exam_3'] = a

df_xgb_all_predictions = df_xgb_all_predictions.join(df3_[['exam_3']])


[0]	validation_0-error:0
Will train until validation_0-error hasn't improved in 10 rounds.
[1]	validation_0-error:0
[2]	validation_0-error:0
[3]	validation_0-error:0
[4]	validation_0-error:0
[5]	validation_0-error:0
[6]	validation_0-error:0
[7]	validation_0-error:0
[8]	validation_0-error:0
[9]	validation_0-error:0
[10]	validation_0-error:0
Stopping. Best iteration:
[0]	validation_0-error:0

[0]	validation_0-error:0
Will train until validation_0-error hasn't improved in 10 rounds.
[1]	validation_0-error:0
[2]	validation_0-error:0
[3]	validation_0-error:0
[4]	validation_0-error:0
[5]	validation_0-error:0
[6]	validation_0-error:0
[7]	validation_0-error:0
[8]	validation_0-error:0
[9]	validation_0-error:0
[10]	validation_0-error:0
Stopping. Best iteration:
[0]	validation_0-error:0

[0]	validation_0-error:0
Will train until validation_0-error hasn't improved in 10 rounds.
[1]	validation_0-error:0
[2]	validation_0-error:0
[3]	validation_0-error:0
[4]	validation_0-error:0
[5]	validation_0-erro

### Gaussian Naive Bayes

In [97]:
i = df_gnb_all_predictions.loc[df_gnb_all_predictions['exam_2'] == 'Não Desistente'].index
df3_ = df3_data_target.iloc[i].copy()
X = df3_[features].values
Y = df3_['target'].values

acur_gnb = []

dict_predict_gnb = {}

# model
classifier_gnb = GaussianNB()

# cv = KFold(n_splits=10,  random_state=2**32 - 1, shuffle=True)
cv = KFold(n_splits=10, shuffle=True)
for train_index, test_index in cv.split(X):

    X_train, X_test, y_train, y_test = X[train_index], X[test_index], Y[train_index], Y[test_index]

    # fit model
    classifier_gnb.fit(X_train, y_train)

    # testing model
    y_predict_gnb = classifier_gnb.predict(X_test)

    # dataframe predictions
    d = dict(zip(test_index, y_predict_gnb))
    dict_predict_gnb.update(d)

    # acum accurary
    acur_gnb.append(accuracy_score(y_test, y_predict_gnb))

print('Acurácia Média Gaussian Naive Bayes', np.mean(acur_gnb))

d = dict(sorted(dict_predict_gnb.items()))
a = np.array([w for v,w in d.items()])
df3_['exam_3'] = a

df_gnb_all_predictions = df_gnb_all_predictions.join(df3_[['exam_3']])


Acurácia Média Gaussian Naive Bayes 0.95


## Treinando com dados até 4º prova

### Random Forest

In [98]:
# selecting all não desistente
i = df_rf_all_predictions.loc[df_rf_all_predictions['exam_3'] == 'Não Desistente'].index
df4_ = df4_data_target.iloc[i].copy()
X = df4_[features].values
Y = df4_['target'].values

# traning Random Forest
acur_rf = []
dict_predict_rf = {}

# model
classifier_rf = RandomForestClassifier(bootstrap=False, criterion='entropy', max_depth=5, n_estimators=25, verbose=0)

# cv = KFold(n_splits=10,  random_state=2**32 - 1, shuffle=True)
cv = KFold(n_splits=10, shuffle=True)
for train_index, test_index in cv.split(X):

    X_train, X_test, y_train, y_test = X[train_index], X[test_index], Y[train_index], Y[test_index]

    # fit model
    classifier_rf.fit(X_train, y_train)

    # testing model
    y_predict_rf  = classifier_rf.predict(X_test)

    # dataframe predictions
    d = dict(zip(test_index, y_predict_rf))
    dict_predict_rf.update(d)

    # acum accurary
    acur_rf.append(accuracy_score(y_test, y_predict_rf))
    
print('Acurácia Média Random Forest:', np.mean(acur_rf))

d = dict(sorted(dict_predict_rf.items()))
a = np.array([w for v,w in d.items()])
df4_['exam_4'] = a

df_rf_all_predictions = df_rf_all_predictions.join(df4_[['exam_4']])


Acurácia Média Random Forest: 0.9800000000000001


### Extra Tree Classifier

In [99]:
i = df_etc_all_predictions.loc[df_etc_all_predictions['exam_3'] == 'Não Desistente'].index
df4_ = df4_data_target.iloc[i].copy()
X = df4_[features].values
Y = df4_['target'].values

acur_etc = []

dict_predict_etc = {}

# model
classifier_etc = ExtraTreesClassifier(bootstrap=False, criterion='entropy', max_depth=3, n_estimators=25)

# cv = KFold(n_splits=10,  random_state=2**32 - 1, shuffle=True)
cv = KFold(n_splits=10, shuffle=True)
for train_index, test_index in cv.split(X):

    X_train, X_test, y_train, y_test = X[train_index], X[test_index], Y[train_index], Y[test_index]

    # fit model
    classifier_etc.fit(X_train, y_train)

    # testing model
    y_predict_etc = classifier_etc.predict(X_test)

    # dataframe predictions
    d = dict(zip(test_index, y_predict_etc))
    dict_predict_etc.update(d)

    # acum accurary
    acur_etc.append(accuracy_score(y_test, y_predict_etc))
    
print('Acurácia Média Extra Tree Classifier:', np.mean(acur_etc))

d = dict(sorted(dict_predict_etc.items()))
a = np.array([w for v,w in d.items()])
df4_['exam_4'] = a

df_etc_all_predictions = df_etc_all_predictions.join(df4_[['exam_4']])


Acurácia Média Extra Tree Classifier: 1.0


### XGBoost

In [100]:
i = df_xgb_all_predictions.loc[df_xgb_all_predictions['exam_3'] == 'Não Desistente'].index
df4_ = df4_data_target.iloc[i].copy()
X = df4_[features].values
Y = df4_['target'].values

acur_xgb = []

dict_predict_xgb = {}

# model
classifier_xgb = XGBClassifier(learning_rate=0.1, max_depth=2, n_estimators=60)

# cv = KFold(n_splits=10,  random_state=2**32 - 1, shuffle=True)
cv = KFold(n_splits=10, shuffle=True)
for train_index, test_index in cv.split(X):

    X_train, X_test, y_train, y_test = X[train_index], X[test_index], Y[train_index], Y[test_index]

    # criando um particao de validação para o modelo xgb
    X_val, y_val = X_test[0:2], y_test[0:2]
    _X_test = X_test[2:]
    _y_test = y_test[2:]

    # fit model
    classifier_xgb.fit(X_train, y_train, early_stopping_rounds=10, eval_set=[(X_val, y_val)])

    # testing model
    y_predict_xgb = classifier_xgb.predict(X_test)

    # dataframe predictions
    d = dict(zip(test_index, y_predict_xgb))
    dict_predict_xgb.update(d)

    # acum accurary
    acur_xgb.append(accuracy_score(y_test, y_predict_xgb))
    
print('Acurácia Média XGBoost', np.mean(acur_xgb))

d = dict(sorted(dict_predict_xgb.items()))
a = np.array([w for v,w in d.items()])
df4_['exam_4'] = a

df_xgb_all_predictions = df_xgb_all_predictions.join(df4_[['exam_4']])


[0]	validation_0-error:0
Will train until validation_0-error hasn't improved in 10 rounds.
[1]	validation_0-error:0
[2]	validation_0-error:0
[3]	validation_0-error:0
[4]	validation_0-error:0
[5]	validation_0-error:0
[6]	validation_0-error:0
[7]	validation_0-error:0
[8]	validation_0-error:0
[9]	validation_0-error:0
[10]	validation_0-error:0
Stopping. Best iteration:
[0]	validation_0-error:0

[0]	validation_0-error:0
Will train until validation_0-error hasn't improved in 10 rounds.
[1]	validation_0-error:0
[2]	validation_0-error:0
[3]	validation_0-error:0
[4]	validation_0-error:0
[5]	validation_0-error:0
[6]	validation_0-error:0.5
[7]	validation_0-error:0.5
[8]	validation_0-error:0.5
[9]	validation_0-error:0.5
[10]	validation_0-error:0.5
Stopping. Best iteration:
[0]	validation_0-error:0

[0]	validation_0-error:0
Will train until validation_0-error hasn't improved in 10 rounds.
[1]	validation_0-error:0
[2]	validation_0-error:0
[3]	validation_0-error:0
[4]	validation_0-error:0
[5]	validat

### Gaussian Naive Bayes

In [101]:
i = df_gnb_all_predictions.loc[df_gnb_all_predictions['exam_3'] == 'Não Desistente'].index
df4_ = df4_data_target.iloc[i].copy()
X = df4_[features].values
Y = df4_['target'].values

acur_gnb = []

dict_predict_gnb = {}

# model
classifier_gnb = GaussianNB()

# cv = KFold(n_splits=10,  random_state=2**32 - 1, shuffle=True)
cv = KFold(n_splits=10, shuffle=True)
for train_index, test_index in cv.split(X):

    X_train, X_test, y_train, y_test = X[train_index], X[test_index], Y[train_index], Y[test_index]

    # fit model
    classifier_gnb.fit(X_train, y_train)

    # testing model
    y_predict_gnb = classifier_gnb.predict(X_test)

    # dataframe predictions
    d = dict(zip(test_index, y_predict_gnb))
    dict_predict_gnb.update(d)

    # acum accurary
    acur_gnb.append(accuracy_score(y_test, y_predict_gnb))

print('Acurácia Média Gaussian Naive Bayes', np.mean(acur_gnb))

d = dict(sorted(dict_predict_gnb.items()))
a = np.array([w for v,w in d.items()])
df4_['exam_4'] = a

df_gnb_all_predictions = df_gnb_all_predictions.join(df4_[['exam_4']])


Acurácia Média Gaussian Naive Bayes 0.95


## Treinando com dados até a 5º prova

### Random Forest

In [102]:
# selecting all não desistente
i = df_rf_all_predictions.loc[df_rf_all_predictions['exam_4'] == 'Não Desistente'].index
df5_ = df5_data_target.iloc[i].copy()
X = df5_[features].values
Y = df5_['target'].values

# traning Random Forest
acur_rf = []
dict_predict_rf = {}

# model
classifier_rf = RandomForestClassifier(bootstrap=False, criterion='entropy', max_depth=5, n_estimators=25, verbose=0)

# cv = KFold(n_splits=10,  random_state=2**32 - 1, shuffle=True)
cv = KFold(n_splits=10, shuffle=True)
for train_index, test_index in cv.split(X):

    X_train, X_test, y_train, y_test = X[train_index], X[test_index], Y[train_index], Y[test_index]

    # fit model
    classifier_rf.fit(X_train, y_train)

    # testing model
    y_predict_rf  = classifier_rf.predict(X_test)

    # dataframe predictions
    d = dict(zip(test_index, y_predict_rf))
    dict_predict_rf.update(d)

    # acum accurary
    acur_rf.append(accuracy_score(y_test, y_predict_rf))
    
print('Acurácia Média Random Forest:', np.mean(acur_rf))

d = dict(sorted(dict_predict_rf.items()))
a = np.array([w for v,w in d.items()])
df5_['exam_5'] = a

df_rf_all_predictions = df_rf_all_predictions.join(df5_[['exam_5']])


Acurácia Média Random Forest: 0.975


### Extra Tree Classifier

In [103]:
i = df_etc_all_predictions.loc[df_etc_all_predictions['exam_4'] == 'Não Desistente'].index
df5_ = df5_data_target.iloc[i].copy()
X = df5_[features].values
Y = df5_['target'].values

acur_etc = []

dict_predict_etc = {}

# model
classifier_etc = ExtraTreesClassifier(bootstrap=False, criterion='entropy', max_depth=3, n_estimators=25)

# cv = KFold(n_splits=10,  random_state=2**32 - 1, shuffle=True)
cv = KFold(n_splits=10, shuffle=True)
for train_index, test_index in cv.split(X):

    X_train, X_test, y_train, y_test = X[train_index], X[test_index], Y[train_index], Y[test_index]

    # fit model
    classifier_etc.fit(X_train, y_train)

    # testing model
    y_predict_etc = classifier_etc.predict(X_test)

    # dataframe predictions
    d = dict(zip(test_index, y_predict_etc))
    dict_predict_etc.update(d)

    # acum accurary
    acur_etc.append(accuracy_score(y_test, y_predict_etc))
    
print('Acurácia Média Extra Tree Classifier:', np.mean(acur_etc))

d = dict(sorted(dict_predict_etc.items()))
a = np.array([w for v,w in d.items()])
df5_['exam_5'] = a

df_etc_all_predictions = df_etc_all_predictions.join(df5_[['exam_5']])


Acurácia Média Extra Tree Classifier: 1.0


### XGBoost

In [104]:
i = df_xgb_all_predictions.loc[df_xgb_all_predictions['exam_4'] == 'Não Desistente'].index
df5_ = df5_data_target.iloc[i].copy()
X = df5_[features].values
Y = df5_['target'].values

acur_xgb = []

dict_predict_xgb = {}

# model
classifier_xgb = XGBClassifier(learning_rate=0.1, max_depth=2, n_estimators=60)

# cv = KFold(n_splits=10,  random_state=2**32 - 1, shuffle=True)
cv = KFold(n_splits=10, shuffle=True)
for train_index, test_index in cv.split(X):

    X_train, X_test, y_train, y_test = X[train_index], X[test_index], Y[train_index], Y[test_index]

    # criando um particao de validação para o modelo xgb
    X_val, y_val = X_test[0:2], y_test[0:2]
    _X_test = X_test[2:]
    _y_test = y_test[2:]

    # fit model
    classifier_xgb.fit(X_train, y_train, early_stopping_rounds=10, eval_set=[(X_val, y_val)])

    # testing model
    y_predict_xgb = classifier_xgb.predict(X_test)

    # dataframe predictions
    d = dict(zip(test_index, y_predict_xgb))
    dict_predict_xgb.update(d)

    # acum accurary
    acur_xgb.append(accuracy_score(y_test, y_predict_xgb))
    
print('Acurácia Média XGBoost', np.mean(acur_xgb))

d = dict(sorted(dict_predict_xgb.items()))
a = np.array([w for v,w in d.items()])
df5_['exam_5'] = a

df_xgb_all_predictions = df_xgb_all_predictions.join(df5_[['exam_5']])


[0]	validation_0-error:0
Will train until validation_0-error hasn't improved in 10 rounds.
[1]	validation_0-error:0
[2]	validation_0-error:0
[3]	validation_0-error:0
[4]	validation_0-error:0
[5]	validation_0-error:0
[6]	validation_0-error:0
[7]	validation_0-error:0
[8]	validation_0-error:0
[9]	validation_0-error:0
[10]	validation_0-error:0
Stopping. Best iteration:
[0]	validation_0-error:0

[0]	validation_0-error:0
Will train until validation_0-error hasn't improved in 10 rounds.
[1]	validation_0-error:0
[2]	validation_0-error:0
[3]	validation_0-error:0
[4]	validation_0-error:0
[5]	validation_0-error:0
[6]	validation_0-error:0
[7]	validation_0-error:0
[8]	validation_0-error:0
[9]	validation_0-error:0
[10]	validation_0-error:0
Stopping. Best iteration:
[0]	validation_0-error:0

[0]	validation_0-error:0
Will train until validation_0-error hasn't improved in 10 rounds.
[1]	validation_0-error:0
[2]	validation_0-error:0
[3]	validation_0-error:0
[4]	validation_0-error:0
[5]	validation_0-erro

### Gaussian Naive Bayes

In [105]:
i = df_gnb_all_predictions.loc[df_gnb_all_predictions['exam_4'] == 'Não Desistente'].index
df5_ = df5_data_target.iloc[i].copy()
X = df5_[features].values
Y = df5_['target'].values

acur_gnb = []

dict_predict_gnb = {}

# model
classifier_gnb = GaussianNB()

# cv = KFold(n_splits=10,  random_state=2**32 - 1, shuffle=True)
cv = KFold(n_splits=10, shuffle=True)
for train_index, test_index in cv.split(X):

    X_train, X_test, y_train, y_test = X[train_index], X[test_index], Y[train_index], Y[test_index]

    # fit model
    classifier_gnb.fit(X_train, y_train)

    # testing model
    y_predict_gnb = classifier_gnb.predict(X_test)

    # dataframe predictions
    d = dict(zip(test_index, y_predict_gnb))
    dict_predict_gnb.update(d)

    # acum accurary
    acur_gnb.append(accuracy_score(y_test, y_predict_gnb))

print('Acurácia Média Gaussian Naive Bayes', np.mean(acur_gnb))

d = dict(sorted(dict_predict_gnb.items()))
a = np.array([w for v,w in d.items()])
df5_['exam_5'] = a

df_gnb_all_predictions = df_gnb_all_predictions.join(df5_[['exam_5']])


Acurácia Média Gaussian Naive Bayes 0.95


## Treinando com dados até 6º prova

### Random Forest

In [106]:
# selecting all não desistente
i = df_rf_all_predictions.loc[df_rf_all_predictions['exam_5'] == 'Não Desistente'].index
df6_ = df6_data_target.iloc[i].copy()
X = df6_[features].values
Y = df6_['target'].values

# traning Random Forest
acur_rf = []
dict_predict_rf = {}

# model
classifier_rf = RandomForestClassifier(bootstrap=False, criterion='entropy', max_depth=5, n_estimators=25, verbose=0)

# cv = KFold(n_splits=10,  random_state=2**32 - 1, shuffle=True)
cv = KFold(n_splits=10, shuffle=True)
for train_index, test_index in cv.split(X):

    X_train, X_test, y_train, y_test = X[train_index], X[test_index], Y[train_index], Y[test_index]

    # fit model
    classifier_rf.fit(X_train, y_train)

    # testing model
    y_predict_rf  = classifier_rf.predict(X_test)

    # dataframe predictions
    d = dict(zip(test_index, y_predict_rf))
    dict_predict_rf.update(d)

    # acum accurary
    acur_rf.append(accuracy_score(y_test, y_predict_rf))
    
print('Acurácia Média Random Forest:', np.mean(acur_rf))

d = dict(sorted(dict_predict_rf.items()))
a = np.array([w for v,w in d.items()])
df6_['exam_6'] = a

df_rf_all_predictions = df_rf_all_predictions.join(df6_[['exam_6']])


Acurácia Média Random Forest: 0.9800000000000001


### Extra Tree Classifier

In [107]:
i = df_etc_all_predictions.loc[df_etc_all_predictions['exam_5'] == 'Não Desistente'].index
df6_ = df6_data_target.iloc[i].copy()
X = df6_[features].values
Y = df6_['target'].values

acur_etc = []

dict_predict_etc = {}

# model
classifier_etc = ExtraTreesClassifier(bootstrap=False, criterion='entropy', max_depth=3, n_estimators=25)

# cv = KFold(n_splits=10,  random_state=2**32 - 1, shuffle=True)
cv = KFold(n_splits=10, shuffle=True)
for train_index, test_index in cv.split(X):

    X_train, X_test, y_train, y_test = X[train_index], X[test_index], Y[train_index], Y[test_index]

    # fit model
    classifier_etc.fit(X_train, y_train)

    # testing model
    y_predict_etc = classifier_etc.predict(X_test)

    # dataframe predictions
    d = dict(zip(test_index, y_predict_etc))
    dict_predict_etc.update(d)

    # acum accurary
    acur_etc.append(accuracy_score(y_test, y_predict_etc))
    
print('Acurácia Média Extra Tree Classifier:', np.mean(acur_etc))

d = dict(sorted(dict_predict_etc.items()))
a = np.array([w for v,w in d.items()])
df6_['exam_6'] = a

df_etc_all_predictions = df_etc_all_predictions.join(df6_[['exam_6']])


Acurácia Média Extra Tree Classifier: 1.0


### XGBoost

In [108]:
i = df_xgb_all_predictions.loc[df_xgb_all_predictions['exam_5'] == 'Não Desistente'].index
df6_ = df6_data_target.iloc[i].copy()
X = df6_[features].values
Y = df6_['target'].values

acur_xgb = []

dict_predict_xgb = {}

# model
classifier_xgb = XGBClassifier(learning_rate=0.1, max_depth=2, n_estimators=60)

# cv = KFold(n_splits=10,  random_state=2**32 - 1, shuffle=True)
cv = KFold(n_splits=10, shuffle=True)
for train_index, test_index in cv.split(X):

    X_train, X_test, y_train, y_test = X[train_index], X[test_index], Y[train_index], Y[test_index]

    # criando um particao de validação para o modelo xgb
    X_val, y_val = X_test[0:2], y_test[0:2]
    _X_test = X_test[2:]
    _y_test = y_test[2:]

    # fit model
    classifier_xgb.fit(X_train, y_train, early_stopping_rounds=10, eval_set=[(X_val, y_val)])

    # testing model
    y_predict_xgb = classifier_xgb.predict(X_test)

    # dataframe predictions
    d = dict(zip(test_index, y_predict_xgb))
    dict_predict_xgb.update(d)

    # acum accurary
    acur_xgb.append(accuracy_score(y_test, y_predict_xgb))
    
print('Acurácia Média XGBoost', np.mean(acur_xgb))

d = dict(sorted(dict_predict_xgb.items()))
a = np.array([w for v,w in d.items()])
df6_['exam_6'] = a

df_xgb_all_predictions = df_xgb_all_predictions.join(df6_[['exam_6']])


[0]	validation_0-error:0
Will train until validation_0-error hasn't improved in 10 rounds.
[1]	validation_0-error:0
[2]	validation_0-error:0
[3]	validation_0-error:0
[4]	validation_0-error:0
[5]	validation_0-error:0
[6]	validation_0-error:0
[7]	validation_0-error:0
[8]	validation_0-error:0
[9]	validation_0-error:0
[10]	validation_0-error:0
Stopping. Best iteration:
[0]	validation_0-error:0

[0]	validation_0-error:0
Will train until validation_0-error hasn't improved in 10 rounds.
[1]	validation_0-error:0
[2]	validation_0-error:0
[3]	validation_0-error:0
[4]	validation_0-error:0
[5]	validation_0-error:0
[6]	validation_0-error:0
[7]	validation_0-error:0
[8]	validation_0-error:0
[9]	validation_0-error:0
[10]	validation_0-error:0
Stopping. Best iteration:
[0]	validation_0-error:0

[0]	validation_0-error:0
Will train until validation_0-error hasn't improved in 10 rounds.
[1]	validation_0-error:0
[2]	validation_0-error:0
[3]	validation_0-error:0
[4]	validation_0-error:0
[5]	validation_0-erro

### Gaussian Naive Bayes

In [109]:
i = df_gnb_all_predictions.loc[df_gnb_all_predictions['exam_5'] == 'Não Desistente'].index
df6_ = df6_data_target.iloc[i].copy()
X = df6_[features].values
Y = df6_['target'].values

acur_gnb = []

dict_predict_gnb = {}

# model
classifier_gnb = GaussianNB()

# cv = KFold(n_splits=10,  random_state=2**32 - 1, shuffle=True)
cv = KFold(n_splits=10, shuffle=True)
for train_index, test_index in cv.split(X):

    X_train, X_test, y_train, y_test = X[train_index], X[test_index], Y[train_index], Y[test_index]

    # fit model
    classifier_gnb.fit(X_train, y_train)

    # testing model
    y_predict_gnb = classifier_gnb.predict(X_test)

    # dataframe predictions
    d = dict(zip(test_index, y_predict_gnb))
    dict_predict_gnb.update(d)

    # acum accurary
    acur_gnb.append(accuracy_score(y_test, y_predict_gnb))

print('Acurácia Média Gaussian Naive Bayes', np.mean(acur_gnb))

d = dict(sorted(dict_predict_gnb.items()))
a = np.array([w for v,w in d.items()])
df6_['exam_6'] = a

df_gnb_all_predictions = df_gnb_all_predictions.join(df6_[['exam_6']])


Acurácia Média Gaussian Naive Bayes 0.95


In [110]:
df_rf_all_predictions

,target,exam_1,exam_2,exam_3,exam_4,exam_5,exam_6
0,Desistente,Desistente,NaN,NaN,NaN,NaN,NaN
1,Desistente,Não Desistente,Não Desistente,Não Desistente,Não Desistente,Não Desistente,Não Desistente
2,Não Desistente,Não Desistente,Não Desistente,Não Desistente,Não Desistente,Não Desistente,Não Desistente
3,Desistente,Desistente,NaN,NaN,NaN,NaN,NaN
4,Desistente,Desistente,NaN,NaN,NaN,NaN,NaN
5,Não Desistente,Não Desistente,Não Desistente,Não Desistente,Não Desistente,Não Desistente,Não Desistente
6,Não Desistente,Não Desistente,Não Desistente,Não Desistente,Não Desistente,Não Desistente,Não Desistente
7,Não Desistente,Não Desistente,Não Desistente,Não Desistente,Não Desistente,Não Desistente,Não Desistente
8,Não Desistente,Não Desistente,Não Desistente,Não Desistente,Não Desistente,Não Desistente,Não Desistente
9,Não Desistente,Não Desistente,Não Desistente,Não Desistente,Não Desistente,Não Desistente,Não Desistente


In [111]:
df_etc_all_predictions

,target,exam_1,exam_2,exam_3,exam_4,exam_5,exam_6
0,Desistente,Desistente,NaN,NaN,NaN,NaN,NaN
1,Desistente,Desistente,NaN,NaN,NaN,NaN,NaN
2,Não Desistente,Não Desistente,Não Desistente,Não Desistente,Não Desistente,Não Desistente,Não Desistente
3,Desistente,Desistente,NaN,NaN,NaN,NaN,NaN
4,Desistente,Desistente,NaN,NaN,NaN,NaN,NaN
5,Não Desistente,Não Desistente,Não Desistente,Não Desistente,Não Desistente,Não Desistente,Não Desistente
6,Não Desistente,Não Desistente,Não Desistente,Não Desistente,Não Desistente,Não Desistente,Não Desistente
7,Não Desistente,Não Desistente,Não Desistente,Não Desistente,Não Desistente,Não Desistente,Não Desistente
8,Não Desistente,Não Desistente,Não Desistente,Não Desistente,Não Desistente,Não Desistente,Não Desistente
9,Não Desistente,Não Desistente,Não Desistente,Não Desistente,Não Desistente,Não Desistente,Não Desistente


In [112]:
df_xgb_all_predictions

,target,exam_1,exam_2,exam_3,exam_4,exam_5,exam_6
0,Desistente,Desistente,NaN,NaN,NaN,NaN,NaN
1,Desistente,Não Desistente,Não Desistente,Não Desistente,Desistente,NaN,NaN
2,Não Desistente,Não Desistente,Não Desistente,Não Desistente,Não Desistente,Não Desistente,Não Desistente
3,Desistente,Desistente,NaN,NaN,NaN,NaN,NaN
4,Desistente,Desistente,NaN,NaN,NaN,NaN,NaN
5,Não Desistente,Não Desistente,Não Desistente,Não Desistente,Não Desistente,Não Desistente,Não Desistente
6,Não Desistente,Não Desistente,Não Desistente,Não Desistente,Desistente,NaN,NaN
7,Não Desistente,Não Desistente,Não Desistente,Não Desistente,Não Desistente,Não Desistente,Não Desistente
8,Não Desistente,Não Desistente,Não Desistente,Não Desistente,Não Desistente,Não Desistente,Não Desistente
9,Não Desistente,Não Desistente,Não Desistente,Não Desistente,Não Desistente,Não Desistente,Não Desistente


In [113]:
df_gnb_all_predictions

,target,exam_1,exam_2,exam_3,exam_4,exam_5,exam_6
0,Desistente,Desistente,NaN,NaN,NaN,NaN,NaN
1,Desistente,Desistente,NaN,NaN,NaN,NaN,NaN
2,Não Desistente,Não Desistente,Não Desistente,Não Desistente,Não Desistente,Não Desistente,Não Desistente
3,Desistente,Desistente,NaN,NaN,NaN,NaN,NaN
4,Desistente,Desistente,NaN,NaN,NaN,NaN,NaN
5,Não Desistente,Não Desistente,Não Desistente,Não Desistente,Não Desistente,Não Desistente,Não Desistente
6,Não Desistente,Não Desistente,Não Desistente,Não Desistente,Não Desistente,Não Desistente,Não Desistente
7,Não Desistente,Não Desistente,Não Desistente,Não Desistente,Não Desistente,Não Desistente,Não Desistente
8,Não Desistente,Não Desistente,Não Desistente,Não Desistente,Não Desistente,Não Desistente,Não Desistente
9,Não Desistente,Desistente,NaN,NaN,NaN,NaN,NaN


## Exportando o resultado para um excel

In [114]:
from pandas import ExcelWriter

writer = ExcelWriter('/content/drive/Shared drives/iniciacao_cientifica_2019_2020/dataframes_experimento/train_no_D.xlsx')
df_rf_all_predictions.to_excel(writer, 'random_forest')
df_etc_all_predictions.to_excel(writer, 'extra_tree_classifier')
df_xgb_all_predictions.to_excel(writer, 'xgboost')
df_gnb_all_predictions.to_excel(writer, 'gaussian_naive_bayes')

writer.save()